# YouTube Corpus videos downloader 2023

## Libraries 

In [4]:
import pandas as pd
from pytube import YouTube
from pytube.exceptions import VideoUnavailable
import os
import time 
from tinydb import TinyDB, Query
# pytubefix es una repo de GitHub que busca arreglar los errores de Pytube
# resultó útil para las transcripciones
from pytubefix import Youtube as yt2

In [6]:
audios_dir = os.path.join(os.getcwd(), "audios")

To do:
- validar si ya se descargo o no 
- subir audios a una bd para no descargar de yt (maybe)
- crear queries de busqueda
- transcribir el audio (subtitulos)
    - whisper (leer, utilizar)
    - cambiar si es necesario .mp3
    - guardar como archivo o directo a bd 

## Create dataframe

In [7]:
df = pd.read_csv("corpus_yt_2023.csv")
df.rename(str.strip, axis="columns", inplace = True)
df.loc[:, "Nombre"].ffill(limit=2, inplace = True)
df.loc[:, "Estado"].ffill(inplace=True)
df.loc[:, "Edad"].ffill(limit=2, inplace = True)
df.loc[:, "Género"].ffill(limit=2, inplace = True)

df.head(20)

,Estado,Nombre,Edad,Género,Título,Fecha de creación,Link,Fecha de recopilación,Aportación
0,Aguascalientes,Alejandra García,26.0,Femenino,REGALO COMÍDA en la CALLE | lo logramos 🙏🏻🥳 Al...,31 agosto 2023,https://youtu.be/hWKy7Ns1pNo?si=CtYGM3G0f_TtUXUz,18 septiembre 2023,Heili
1,Aguascalientes,Alejandra García,26.0,Femenino,"habitación principal | REMODELACIÓN, CON POQUI...",30 noviembre 2022,https://youtu.be/bm2xz0_HrGc?si=anegbcnciRTpv535,18 septiembre 2023,Heili
2,Aguascalientes,Alejandra García,26.0,Femenino,Nuestro Árbol de Navidad Casero | Ale García ✨,21 noviembre 2021,https://youtu.be/JT3WoDrrl1A?si=U7yvebArA8Qx6wkv,18 septiembre 2023,Heili
3,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Database 

In [9]:
db_file = "audio_database.json"
DOWNLOADED = [] # rows of the audios that are already downloaded

try:
    with open(db_file, 'r'):
        pass
    db = TinyDB(db_file)
except FileNotFoundError:
    db = TinyDB(db_file)


# Guardamos: video_id, estado, autor, edad, genero, ruta
def save_audio_index(index_downloaded):
    with open("downloaded.txt", 'a') as file:
        file.write(str(index_downloaded) + '\n')

def create_db_entries(df_entry, r):
    id = YouTube(df_entry["Link"]).video_id
    db.insert({"Estado":df_entry["Estado"], "Autor":df_entry["Nombre"], "Edad":df_entry["Edad"], 
               "Genero":df_entry["Género"], "VideoID":id, "Ruta":r})

def load_downloaded_audios():
    # archivo txt con los indices de los auidos ya descargados
    try:
        with open("downloaded.txt", "r") as file:
            for row_num in file:
                val = int(row_num.strip("\n"))
                DOWNLOADED.append(val)
    except FileNotFoundError as e:
        print(f"File not found: {e}")

def errase_db():
    db.truncate()
    db.all()

def errase_downloaded_txt():
    try:
        with open("downloaded.txt", "w") as _:
           pass
    except FileNotFoundError as e:
        print(f'File not found: {e}')

## Download audios

In [10]:
def download_audio(link, row, i):
    
    try: 
        yt_obj = YouTube(link)
    except VideoUnavailable:
        print(f'Video at {link} is unavailable, skipping...')
    else:
        try:
            audio = yt_obj.streams.last()
            file_name = f'{yt_obj.video_id}.webm'
            path = os.path.join(audios_dir, file_name)
            audio.download(filename=file_name, output_path=audios_dir, timeout=60)
            save_audio_index(i) 
            create_db_entries(row, path)
            print(f'Downloaded {yt_obj.title[0:20] + "..."}')
        except Exception as e:
            print(f'An error occurred while downloading the video: {str(e)}')


## Main

In [6]:
downloaded_counter = 0 

df_available = df.dropna()
available_total = len(df_available)
load_downloaded_audios()

for i, row in df_available.iterrows():

    link = row["Link"]

    if downloaded_counter % 5 == 0 and downloaded_counter != 0: 
        # Cada 5 descargas esperamos un minuto para evitar ser bloqueados
        time.sleep(60)

    if i in DOWNLOADED:
        print("Already downloaded. Skiping...")
        continue
    else:
        download_audio(link, row, i)

    downloaded_counter += 1


print(f"\n\nFinished downloading. {downloaded_counter} of {available_total} videos downloaded.")

Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
An error occurred while downloading the video: QbRt-eZwuno is age restricted, and can't be accessed without logging in.
Already downloaded. Skiping...
Already downloaded. Skiping...
Already downloaded. Skiping...
Already down

In [7]:
#errase_db()
#errase_downloaded_txt()

In [ ]:
db.all()

[{'Estado': 'Aguascalientes',
  'Autor': 'Alejandra García ',
  'Edad': 26.0,
  'Genero': 'Femenino',
  'VideoID': 'hWKy7Ns1pNo',
  'Ruta': 'c:\\Users\\ruyca\\Desktop\\UNAM\\2024-1\\IIMAS-servicio\\codigos\\corpus_downloader\\audios\\hWKy7Ns1pNo.webm'},
 {'Estado': 'Aguascalientes',
  'Autor': 'Alejandra García ',
  'Edad': 26.0,
  'Genero': 'Femenino',
  'VideoID': 'bm2xz0_HrGc',
  'Ruta': 'c:\\Users\\ruyca\\Desktop\\UNAM\\2024-1\\IIMAS-servicio\\codigos\\corpus_downloader\\audios\\bm2xz0_HrGc.webm'},
 {'Estado': 'Aguascalientes',
  'Autor': 'Alejandra García ',
  'Edad': 26.0,
  'Genero': 'Femenino',
  'VideoID': 'JT3WoDrrl1A',
  'Ruta': 'c:\\Users\\ruyca\\Desktop\\UNAM\\2024-1\\IIMAS-servicio\\codigos\\corpus_downloader\\audios\\JT3WoDrrl1A.webm'},
 {'Estado': 'Baja California',
  'Autor': 'Carolina Vázquez Díaz',
  'Edad': 29.0,
  'Genero': 'Femenino',
  'VideoID': 'H9zhwMHfAzA',
  'Ruta': 'c:\\Users\\ruyca\\Desktop\\UNAM\\2024-1\\IIMAS-servicio\\codigos\\corpus_downloader\\audios\

# Busqueda segun atributos

In [1]:
#### DIFERENTES QUERIES

def state_equals(estado):
    Video = Query()
    return db.search(Video.Estado == estado)

def age_equals(edad):
    Video = Query()
    return db.search(Video.Edad == edad)

def age_is_more(num=18):
    # Busca por edad que sea mayor o igual a un numero 
    Video = Query()
    return db.search(Video.Edad >= num)

def age_is_less(num=18):
    Video = Query()
    return db.search(Video.Edad <= num)

def gender_is(genero):
    # Busca por genero, feminino o masculino
    Video = Query()
    return db.search(Video.Genero == genero)


In [11]:
##### PARA REALIZAR BUSQUEDAS ########
print("*"*10)

**********


In [8]:
print(state_equals("Baja California"))
print(age_equals(36))
print(age_is_more(18))
print(age_is_less(50))
print(gender_is("Masculino"))


[{'Estado': 'Baja California', 'Autor': 'Carolina Vázquez Díaz', 'Edad': 29.0, 'Genero': 'Femenino', 'VideoID': 'H9zhwMHfAzA', 'Ruta': 'c:\\Users\\ruyca\\Desktop\\UNAM\\2024-1\\IIMAS-servicio\\codigos\\corpus_downloader\\audios\\H9zhwMHfAzA.webm'}, {'Estado': 'Baja California', 'Autor': 'Carolina Vázquez Díaz', 'Edad': 29.0, 'Genero': 'Femenino', 'VideoID': 'kS_in6b-nxA', 'Ruta': 'c:\\Users\\ruyca\\Desktop\\UNAM\\2024-1\\IIMAS-servicio\\codigos\\corpus_downloader\\audios\\kS_in6b-nxA.webm'}, {'Estado': 'Baja California', 'Autor': 'Carolina Vázquez Díaz', 'Edad': 29.0, 'Genero': 'Femenino', 'VideoID': 'MtIPR7anf98', 'Ruta': 'c:\\Users\\ruyca\\Desktop\\UNAM\\2024-1\\IIMAS-servicio\\codigos\\corpus_downloader\\audios\\MtIPR7anf98.webm'}, {'Estado': 'Baja California', 'Autor': 'Kimberly Shantal Adame', 'Edad': 30.0, 'Genero': 'Femenino', 'VideoID': 'QvcnM_Yx1q0', 'Ruta': 'c:\\Users\\ruyca\\Desktop\\UNAM\\2024-1\\IIMAS-servicio\\codigos\\corpus_downloader\\audios\\QvcnM_Yx1q0.webm'}, {'Esta

## Conseguir la ruta de los audios segun ciertos criterios
-----
Para poder ubicar la ruta de los audios según un criterio de búsqueda, considera los siguientes atributos de búsqueda: 

1. **Estado**: entidad federativa del hablante.
2. **Autor**: nombre del hablante. 
3. **Edad**: edad del hablante.
4. **Género**: género del hablante (masculino o femenino).

### Tipos de búsqueda
#### 1. Búsqueda simple
Se utiliza cuando se desea hacer una búsqueda utilizando un solo atributo, por ejemplo: 
- Encontrar audios del **Estado** de Baja California.
- Buscar todos los audios cuya **edad** del hablante sea mayor o igual que 18. 
- Conseguir todos los audios del **género** masculino.

#### 2. Búsqueda compuesta
Se utiliza cuando se desea realizar una búsqueda combinando varios atributos, por ejemplo: 
- Encontrar audios del **estado** de Baja California cuyos hablantes sean del **género** masculino.


El siguiente bloque de código pregunta por estos dos tipos de búsqueda para poder filtrar la búsqueda

In [9]:
### IMPRIMIR RUTAS SEGUN MATCH ###

print("Selecciona los atributos por los que buscas filtrar (donde 1 significa que deseas realizar una busqueda \
      utilizando ese atributo)")

busqueda_por_edad = int(input("Por edad" "))


while True:
    tipo_busqueda = int(input("Escribe el número de busqueda a realizar:\n1. Búsqueda simple\n2. Búsqueda compuesta"))
    if tipo_busqueda == 1:
        num_atributo = int(input("Selecciona el atributo a buscar:\n1. Estado\n2. Autor\n \
                                 3. Edad\n4. Genero"))
        if True:
            pass
    elif tipo_busqueda == 2:
        pass
    else:
        print("Opcion invalida, escoja de nuevo.")





matches = state_equals("Baja California")
for cont, video in enumerate(matches):
    print(f"{cont+1}/{len(matches)} - {video['Ruta']}")

Selecciona los atributos por los que buscas filtrar (donde 1 significa que deseas realizar una busqueda       utilizando ese atributo)


ValueError: invalid literal for int() with base 10: ''